In [1]:
import pandas as pd
import torch
from torch import nn
import numpy as np
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import f1_score
import random
import time

/home/wujh1123/anaconda3/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = pd.read_csv('dataset/train.tsv', sep='\t')
val_data = pd.read_csv('dataset/val.tsv', sep='\t')
test_data = pd.read_csv('dataset/test.tsv', sep='\t')
train_data = pd.concat([train_data, val_data], axis=0)
train_data

,id,source,utterance,classes
0,356,6,Start by reading the preliminary information,OTHER
1,357,6,"""Hello Ms. Klein, I am responsible for you in ...",OTHER
2,358,6,Since yesterday afternoon?,OTHER
3,359,6,"Hmm, but the shortness of breath only came whi...",OTHER
4,360,6,Have you ever laid down during the day when yo...,OTHER
...,...,...,...,...
402,1526,23,Is this the right career for you? Do you have ...,SF
403,1527,23,Your nightmares aren't about your work either?...,OTHER
404,1528,23,But that started at some point? So it's been m...,OTHER
405,1529,23,Are there any illnesses in your family?,SF


In [3]:
num_classes = 6 
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tag_list=["AM","MS","OTHER","PH","SF","SR"]
tag_to_idx={}
for i in range(len(tag_list)):
    tag_to_idx[tag_list[i]]=i
    
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        utterance = str(self.data.iloc[idx]['utterance'])
        labels = self.data.iloc[idx]['classes'] 
        labels = self.label_encoded(labels)
        encoding = self.tokenizer.encode_plus(
            utterance,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float32)
        }
    def label_encoded(self,labels):
        encode_label = np.zeros(6,dtype=int)
        list_label = labels.split(",")
        for i in list_label:
            idx = tag_to_idx[i]
            encode_label[idx] = 1
        return encode_label 

max_len = 256
batch_size = 8

train_dataset = CustomDataset(train_data, tokenizer, max_len)
val_dataset = CustomDataset(val_data, tokenizer, max_len)



train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



/home/wujh1123/anaconda3/envs/nlp/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [5]:
from transformers import  get_cosine_schedule_with_warmup
from torch.optim import AdamW 
def init_model(model,epochs):
    model.to(device)
    optimizer = AdamW(model.parameters(),lr=2e-5,eps=1e-6,betas=(0.9, 0.999))
    total_steps = len(train_dataloader) * epochs
    scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=500,num_training_steps=total_steps)
    return model , optimizer , scheduler

In [6]:
weights = [1,1,0.2,1,1,1]
weights = torch.tensor(weights).to(device)
loss_fn = nn.BCEWithLogitsLoss()
def evalute(dataloader,model):
    model.eval()
    y_pred = []
    y_target = []
    with torch.no_grad():
        for _, batch in enumerate(dataloader):
            b_input_ids = batch['input_ids'].to(device)
            b_attn_mask = batch['attention_mask'].to(device)
            b_labels = batch['labels'].to(device)
            logits = model(b_input_ids, b_attn_mask)
            y_pred.extend(torch.sigmoid(logits.logits).cpu().detach().numpy().tolist())         
            y_target.extend(b_labels.cpu().detach().numpy().tolist())
    y_preds = (np.array(y_pred)>0.5).astype(int)
    marco_f1= f1_score(y_target,y_preds,average='macro')
    # print("marco f1 score : ",marco_f1)
    return marco_f1
def train(model,train_dataloader,val_dataloader,optimizer,scheduler,path,epochs,evaluation):
    max_score = 0
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'F1 score':^9} | {'Elapsed':^9}")
    for epoch_i in range(epochs):
        t0_epoch = time.time()
        total_loss = 0
        for step, batch in enumerate(train_dataloader):
            model.train()
            input_ids = batch['input_ids'].to(device)
            attn_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attn_mask, labels=labels)
            logits = outputs.logits
            loss = loss_fn(logits, labels)
            total_loss += loss.item()
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()    
            model.zero_grad()
            
        avg_train_loss = total_loss / len(train_dataloader)
        if evaluation == True:
            time_elapsed = time.time() - t0_epoch
            score = evalute(val_dataloader,model)
            if score > max_score:
                torch.save(model.state_dict(),path)
                # print('save model')
                max_score = score
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {score:^9.6f} | {time_elapsed:^9.2f}")
        print("")
    print("best score: ",max_score)



In [7]:
model_list = []
for i in range(15):
    epochs = 10
    model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=num_classes)
    bert_classifier, optimizer, scheduler = init_model(model,epochs=epochs)
    path = f"w_weight{i}.pth"
    train(bert_classifier, train_dataloader, val_dataloader,optimizer,scheduler ,path,epochs=epochs, evaluation=True)
    model_list.append(model)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 Epoch  |  Train Loss  | F1 score  |  Elapsed 
   1    |   0.455426   | 0.359184  |   25.30  

   2    |   0.267134   | 0.579066  |   25.12  

   3    |   0.186373   | 0.802905  |   25.12  

   4    |   0.139381   | 0.857821  |   24.91  

   5    |   0.112082   | 0.893845  |   24.87  

   6    |   0.082744   | 0.929405  |   24.99  

   7    |   0.067112   | 0.950980  |   24.99  

   8    |   0.056745   | 0.958547  |   25.04  

   9    |   0.050904   | 0.959204  |   25.02  

  10    |   0.047139   | 0.963462  |   25.09  

best score:  0.9634623000657211


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 Epoch  |  Train Loss  | F1 score  |  Elapsed 
   1    |   0.445409   | 0.153328  |   25.08  

   2    |   0.280010   | 0.697994  |   25.08  

   3    |   0.190338   | 0.810380  |   25.08  

   4    |   0.139049   | 0.869351  |   25.05  

   5    |   0.101937   | 0.896669  |   24.66  

   6    |   0.080233   | 0.945969  |   25.07  

   7    |   0.064078   | 0.962625  |   25.04  

   8    |   0.055440   | 0.962510  |   25.11  

   9    |   0.050203   | 0.964070  |   25.02  

  10    |   0.045236   | 0.961114  |   24.39  

best score:  0.9640699658605593


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

 Epoch  |  Train Loss  | F1 score  |  Elapsed 
   1    |   0.454921   | 0.115717  |   25.11  

   2    |   0.290890   | 0.636284  |   25.03  

   3    |   0.196743   | 0.775691  |   25.08  

   4    |   0.153940   | 0.840033  |   25.09  

   5    |   0.120065   | 0.860979  |   25.05  

   6    |   0.091093   | 0.906707  |   24.98  

   7    |   0.069914   | 0.941420  |   24.99  

   8    |   0.060896   | 0.952266  |   25.03  

   9    |   0.052625   | 0.955804  |   24.78  

  10    |   0.049422   | 0.957530  |   24.70  

best score:  0.9575303385887938


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 Epoch  |  Train Loss  | F1 score  |  Elapsed 
   1    |   0.458347   | 0.120459  |   24.97  

   2    |   0.289441   | 0.528455  |   24.89  

   3    |   0.210908   | 0.774943  |   25.11  

   4    |   0.162373   | 0.845530  |   25.13  

   5    |   0.119272   | 0.881308  |   25.12  

   6    |   0.091046   | 0.906263  |   25.14  

   7    |   0.078660   | 0.940096  |   25.07  

   8    |   0.065308   | 0.947408  |   25.14  

   9    |   0.056325   | 0.951958  |   25.13  

  10    |   0.052300   | 0.951958  |   25.00  

best score:  0.9519579009409477


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 Epoch  |  Train Loss  | F1 score  |  Elapsed 
   1    |   0.450514   | 0.174143  |   25.03  

   2    |   0.298316   | 0.509090  |   25.03  

   3    |   0.203539   | 0.744770  |   24.80  

   4    |   0.159381   | 0.822267  |   25.21  

   5    |   0.120653   | 0.868048  |   25.14  

   6    |   0.095058   | 0.915715  |   25.14  

   7    |   0.077401   | 0.927647  |   25.17  

   8    |   0.067333   | 0.940945  |   25.06  

   9    |   0.060306   | 0.944802  |   25.07  

  10    |   0.054952   | 0.944802  |   25.08  

best score:  0.9448023213641658


In [7]:
def essem_evalute(dataloader,model_list):
    for i in range(len(model_list)):
        model_list[i].eval()
    y_pred = []
    y_target = []
    with torch.no_grad():
        for step, batch in enumerate(dataloader):
            b_input_ids = batch['input_ids'].to(device)
            b_attn_mask = batch['attention_mask'].to(device)
            b_labels = batch['labels'].to(device)
            tmp = 0
            for i in range(len(model_list)):
                logits = model_list[i](b_input_ids, b_attn_mask)
                tmp += torch.sigmoid(logits.logits)
            tmp = tmp / len(model_list)
            y_pred.extend(tmp.cpu().detach().numpy().tolist())         
            y_target.extend(b_labels.cpu().detach().numpy().tolist())
    y_preds = (np.array(y_pred)>0.5).astype(int)
    marco_f1= f1_score(y_target,y_preds,average='macro')
    # print("marco f1 score : ",marco_f1)
    return marco_f1
print(essem_evalute(val_dataloader,model_list))

NameError: name 'model_list' is not defined

In [8]:
class TestDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        ids = self.data.iloc[idx]['id']
        utterance = str(self.data.iloc[idx]['utterance'])
        encoding = self.tokenizer.encode_plus(
            utterance,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'id' : ids,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
test_dataset = TestDataset(test_data, tokenizer, max_len)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [9]:
prdict_model_list=[]
for i in range(15):
    path = f'./w_weight{i}.pth'
    bert_classifier = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=num_classes).to(device)
    bert_classifier.load_state_dict(torch.load(path))
    prdict_model_list.append(bert_classifier)
    bert_classifier.eval()
y_pred = []
ids = []
with torch.no_grad():
    for batch  in test_dataloader:
        id_0 = batch['id'].cpu().item()
        ids.append(id_0)
        b_input_ids = batch['input_ids'].to(device)
        b_attn_mask = batch['attention_mask'].to(device)
        tmp = 0
        for i in range(len(prdict_model_list)):
            logits = prdict_model_list[i](b_input_ids, b_attn_mask)
            
            tmp += torch.sigmoid(logits.logits)
        tmp = tmp / len(prdict_model_list)
        y_pred.extend(tmp.cpu().detach().numpy().tolist())  

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

In [14]:
y_preds = (np.array(y_pred) > 0.6).astype(int)
# y_preds

In [15]:
df = pd.DataFrame(y_preds, columns=tag_list)
df_id =pd.DataFrame(ids, columns=["id"])
merged_df = pd.concat([df_id, df], axis=1)
# merged_df

In [16]:
data_rows = merged_df.to_dict(orient='records')

In [17]:
import csv
with open("submission.csv", 'w', newline='') as csvfile:
    fieldnames=["id","AM","MS","OTHER","PH","SF","SR"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data_rows)